<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Severino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import copy
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

# Classe Task
Viene definita una classe `Task`, che viene inizializzata a partire da un `DataFrame` e ha come attributi `X`, `y` e le loro suddivisioni per il training e il testing.

La classe contiene anche un metodo per scalare l'`X` con uno specifico scaler.

In [2]:
class Task:
  def __init__(self, df):
    X = df.drop("Label", axis = 1)
    X = X.drop("Task", axis = 1)
    X = X.drop("Id", axis = 1)
    y = df["Label"]

    self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

  def scale(self, sc):
    sc.fit(self.X_train)
    self.X_train = sc.transform(self.X_train)
    self.X_test = sc.transform(self.X_test)

Di seguito vengono letti i `csv` dei Task già ripuliti e vengono salvati come oggetti `Task` all'interno dell'array `tasks`.

In [3]:
tasks = []
for i in range(11, 22):
  df = pd.read_csv("Task_{}_clean.csv".format(i))
  task = Task(df)
  tasks.append(task)

Di seguito viene calcolata la moda delle lable del set di testing e viene definita una funzione per calcolarne la somiglianza con i risultati delle predizioni.

In [4]:
y_tests = []
for task in tasks:
  y_tests.append(copy.deepcopy(task.y_test))

y_test_mode, y_test_count = stats.mode(y_tests)

def similarity(predictions_mode):
  if len(y_test_mode) != len(predictions_mode):
    return 0.0

  correct = 0
  for i in range(len(predictions_mode)):
    if y_test_mode[i] == predictions_mode[i]:
      correct += 1

  return correct / len(predictions_mode)

# Funzione `test_boosting`
Dopo aver eventualmente scalato i `Task` mediante uno scaler, viene eseguito, task per task, un fit del classificatore, e vengono salvati gli score e le predizioni, di cui verranno stampati rispettivamente la media e la moda.

In [5]:
def test_boosting(clf, sc = None):
  test_tasks = copy.deepcopy(tasks)

  if sc is not None:
    for task in test_tasks:
      task.scale(sc)

  cross_val_scores = []
  scores = []
  predictions = []

  for task in test_tasks:
    cross_val_scores.append(cross_val_score(clf, task.X_train, task.y_train, cv = 4, n_jobs = -1).mean())
    clf.fit(task.X_train, task.y_train)
    scores.append(clf.score(task.X_test, task.y_test))
    predictions.append(clf.predict(task.X_test))

  print("Results{}:".format("" if sc is None else " (scaled tasks)"))
  print(" - Mean cross_val_score: {:.2%}".format(np.mean(cross_val_scores)))
  print(" - Mean score:           {:.2%}".format(np.mean(scores)))

  mode, count = stats.mode(predictions)
  print(" - Similarity:           {:.2%}\n".format(similarity(mode)))

# Classificatore `AdaBoost`
Un classificatore AdaBoost è un meta-stimatore che inizia inserendo un classificatore nel set di dati originale e quindi adatta copie aggiuntive del classificatore nello stesso set di dati, ma dove i pesi delle istanze classificate in modo errato vengono regolati in modo tale che i classificatori successivi si concentrino maggiormente sui casi difficili.

In [6]:
clf = AdaBoostClassifier(n_estimators = 100, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 76.40%
 - Mean score:           78.03%
 - Similarity:           79.17%



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 79.17%
 - Mean score:           79.17%
 - Similarity:           79.17%



# Classificatore Gradient Boosting

Questo algoritmo costruisce un modello additivo in modo graduale; consente l'ottimizzazione di funzioni di perdita differenziabili arbitrarie. In ogni fase, `n_classes_` alberi di regressione sono adattati al gradiente negativo della funzione di perdita, ad es. perdita di log binaria o multiclasse.

In [7]:
clf = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.01, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 83.75%
 - Mean score:           88.26%
 - Similarity:           91.67%



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 86.35%
 - Mean score:           88.26%
 - Similarity:           95.83%



# Albero di classificazione Gradient Boosting basato sugli istogrammi
Questo stimatore ha il supporto nativo per i valori mancanti (`NaN`). Durante l'addestramento, il coltivatore di alberi impara ad ogni punto di divisione se i campioni con valori mancanti devono andare al figlio sinistro o destro, in base al potenziale guadagno. Durante la previsione, i campioni con valori mancanti vengono assegnati di conseguenza al figlio sinistro o destro. Se durante l'addestramento non sono stati rilevati valori mancanti per una determinata funzionalità, i campioni con valori mancanti vengono mappati a qualunque figlio abbia il maggior numero di campioni.

In [8]:
clf = HistGradientBoostingClassifier(learning_rate = 0.01, random_state = 0)
test_boosting(clf)
test_boosting(clf, MinMaxScaler(feature_range = (0, 1)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results:
 - Mean cross_val_score: 82.93%
 - Mean score:           87.12%
 - Similarity:           91.67%



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


Results (scaled tasks):
 - Mean cross_val_score: 82.93%
 - Mean score:           87.12%
 - Similarity:           91.67%



# Classificatore Percettrone multi-strato
Vengono scalati i task tramite uno `StandardScaler`, in seguito viene eseguita una ricerca sui parametri forniti, e viene testato su tutti i task un `MLPCLassifier` con i parametri migliori.

Infine viene raccolta la predizione per ogni sample del testing per ogni task, viene calcolata la moda attraverso tutti i task per ogni sample e viene stampata a schermo.

In [9]:
sc = StandardScaler()

scaled_tasks = copy.deepcopy(tasks)

for task in scaled_tasks:
  task.scale(sc)

param_grid = {
  "hidden_layer_sizes": [(150, 100, 50), (120, 80, 40), (100, 50, 30)],
  "activation": ['identity', 'logistic', 'tanh', 'relu'],
  "alpha": [0.0000001, 0.00001, 0.1],
  "max_iter": [1000, 2000, 3000]
}
search = HalvingGridSearchCV(MLPClassifier(solver = "lbfgs", random_state = 0), param_grid, cv = 4, n_jobs = -1)

cross_val_scores = []
scores = []
predictions = []

for task in scaled_tasks:
  search.fit(task.X_train, task.y_train)

  clf = MLPClassifier(**search.best_params_, solver = "lbfgs", random_state = 0)

  cross_val_scores.append(cross_val_score(clf, task.X_train, task.y_train, cv = 4, n_jobs = -1).mean())
  clf.fit(task.X_train, task.y_train)
  scores.append(clf.score(task.X_test, task.y_test))
  predictions.append(clf.predict(task.X_test))

print("Results (scaled tasks):")
print(" - Mean cross_val_score: {:.2%}".format(np.mean(cross_val_scores)))
print(" - Mean score:           {:.2%}".format(np.mean(scores)))

mode, count = stats.mode(predictions)
print(" - Similarity:           {:.2%}\n".format(similarity(mode)))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

Results (scaled tasks):
 - Mean cross_val_score: 74.36%
 - Mean score:           78.41%
 - Similarity:           66.67%

